In [ ]:
! pip install transformers datasets peft 
! pip install evaluate
! pip install wandb
! pip install jiwer

In [ ]:
#!pip install transformers==4.45.0
! pip install transformers==4.45.0

In [1]:
import torch
print(torch.__version__)  # Shows PyTorch version
print(torch.version.cuda)  # Shows the CUDA version being used

2.7.1+cu126
12.6


In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [2]:
import wandb
wandb.login(key='cd3943d98b7ac4e0a0abd24721f4016a7942166f')

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(
    project="stt-for-tibetan-language",   # e.g., "stt-training"
    entity="stt-for-tibet"
)


In [4]:
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load pre-trained Wav2Vec2 model and processor
model_name = "ganga4364/mms_300_v4.96000"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)


In [5]:
from datasets import load_from_disk

dataset_train = load_from_disk('/data/wav2vec2/train_prepare_dataset.arrow')
dataset_val = load_from_disk('/data/wav2vec2/val_prepare_dataset.arrow')

In [6]:
dataset_train['input_values'][0]

[-0.0017849294235929847,
 -0.002666810993105173,
 -0.0017849294235929847,
 -0.0011970082996413112,
 -0.0023728504311293364,
 -0.0006090872338972986,
 -2.1166113583603874e-05,
 0.00027279442292638123,
 -0.00384265324100852,
 -0.00384265324100852,
 -0.007664139848202467,
 -0.0070762187242507935,
 -0.01119166612625122,
 -0.010897706262767315,
 -0.00825206097215414,
 -0.009427903220057487,
 -0.00501849502325058,
 -0.002666810993105173,
 -2.1166113583603874e-05,
 0.0032123997807502747,
 0.004388242028653622,
 0.004976163152605295,
 0.011443295516073704,
 0.015558742918074131,
 0.014676861464977264,
 0.014382900670170784,
 0.01203121617436409,
 0.010855373926460743,
 0.009679531678557396,
 0.008209729567170143,
 0.009973492473363876,
 0.008503689430654049,
 0.007621807977557182,
 0.003800320904701948,
 0.0035063603427261114,
 0.002918439218774438,
 0.00027279442292638123,
 -0.0017849294235929847,
 -0.0006090872338972986,
 -0.0006090872338972986,
 -0.0017849294235929847,
 -0.00237285043112933

In [7]:
dataset_train[0]

{'input_values': [-0.0017849294235929847,
  -0.002666810993105173,
  -0.0017849294235929847,
  -0.0011970082996413112,
  -0.0023728504311293364,
  -0.0006090872338972986,
  -2.1166113583603874e-05,
  0.00027279442292638123,
  -0.00384265324100852,
  -0.00384265324100852,
  -0.007664139848202467,
  -0.0070762187242507935,
  -0.01119166612625122,
  -0.010897706262767315,
  -0.00825206097215414,
  -0.009427903220057487,
  -0.00501849502325058,
  -0.002666810993105173,
  -2.1166113583603874e-05,
  0.0032123997807502747,
  0.004388242028653622,
  0.004976163152605295,
  0.011443295516073704,
  0.015558742918074131,
  0.014676861464977264,
  0.014382900670170784,
  0.01203121617436409,
  0.010855373926460743,
  0.009679531678557396,
  0.008209729567170143,
  0.009973492473363876,
  0.008503689430654049,
  0.007621807977557182,
  0.003800320904701948,
  0.0035063603427261114,
  0.002918439218774438,
  0.00027279442292638123,
  -0.0017849294235929847,
  -0.0006090872338972986,
  -0.00060908723

In [8]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [9]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [10]:
import evaluate

# Load the CER (Character Error Rate) metric
cer_metric = evaluate.load("cer")

In [11]:
import numpy as np 

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    print(pred_str)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print(label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    print(cer)
    return {"cer": cer}

In [12]:
import numpy as np
import evaluate
from tibetan_wer.metrics import wer as tib_wer, ser as tib_ser

cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    # Convert logits to token IDs using argmax (this was missing)
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    
    label_ids = pred.label_ids
    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids)  # Use the processor's batch_decode, not tokenizer's
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    tib_wer_res = tib_wer(predictions=pred_str, references=label_str)
    tib_ser_res = tib_ser(predictions=pred_str, references=label_str)

    macro_wer = tib_wer_res['macro_wer']
    micro_wer = tib_wer_res['micro_wer']
    word_subs = tib_wer_res['substitutions']
    word_ins = tib_wer_res['insertions']
    word_dels = tib_wer_res['deletions']

    macro_ser = tib_ser_res['macro_ser']
    micro_ser = tib_ser_res['micro_ser']
    syl_subs = tib_ser_res['substitutions']
    syl_ins = tib_ser_res['insertions']
    syl_dels = tib_ser_res['deletions']

    return {"cer": cer,
            "tib_macro_wer": macro_wer,
            "tib_micro_wer": micro_wer,
            "word_substitutions": word_subs,
            "word_insertions":word_ins,
            "word_deletions":word_dels,
            "tib_macro_ser": macro_ser,
            "tib_micro_ser": micro_ser,
            "syllable_substitutions": syl_subs,
            "syllable_insertions": syl_ins,
            "syllable_deletions": syl_dels
            }

In [ ]:
#test training
training_args = TrainingArguments(
        output_dir="./specific_speaker_model",
        eval_strategy="steps",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,
        learning_rate=3e-4,
        num_train_epochs=50,
        warmup_steps=500,
        logging_steps=100,
        eval_steps=100,
        save_steps=100,
        fp16=True,
        save_total_limit=50,
        report_to="wandb",
    )



In [ ]:
#training arguments
training_args = TrainingArguments(
        output_dir="./specific_speaker_model",
        eval_strategy="steps",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,
        learning_rate=3e-4,
        num_train_epochs=100,
        warmup_steps=500,
        logging_steps=100,
        eval_steps=1000,
        save_steps=1000,
        fp16=True,
        save_total_limit=50,
        dataloader_num_workers=4,
        report_to="wandb",
    )

In [14]:
model.freeze_feature_encoder()

In [15]:
model.config.ctc_zero_infinity = True

In [16]:
  # Create the Trainer object
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=processor.feature_extractor
)

/tmp/ipykernel_15816/1331374303.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:

# Fine-tune the model
#trainer.train()
trainer.train(resume_from_checkpoint=True)


/workspace/stt-split-audio/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Cer,Tib Macro Wer,Tib Micro Wer,Word Substitutions,Word Insertions,Word Deletions,Tib Macro Ser,Tib Micro Ser,Syllable Substitutions,Syllable Insertions,Syllable Deletions
3600,0.072500,1.678472,0.244235,0.526051,0.503666,474,135,78,0.473008,0.465252,520,66,50
3700,0.077700,1.680636,0.242964,0.516263,0.492669,472,140,60,0.461990,0.452085,507,49,62
3800,0.071500,1.714588,0.245687,0.543177,0.510264,480,145,71,0.489102,0.471836,526,61,58
3900,0.068400,1.754458,0.247322,0.544399,0.511730,496,142,60,0.482887,0.468910,524,68,49
4000,0.059600,1.738715,0.238787,0.519820,0.493402,471,132,70,0.476364,0.461595,510,76,45
4100,0.068800,1.766872,0.240421,0.541137,0.505865,478,149,63,0.481112,0.465252,522,58,56
4200,0.054800,1.809397,0.247685,0.534879,0.510997,482,155,60,0.484980,0.471105,533,56,55
4300,0.053700,1.779802,0.245324,0.526058,0.494868,475,137,63,0.475693,0.460132,512,58,59
4400,0.046800,1.743386,0.238969,0.515546,0.486804,466,131,67,0.470107,0.452816,504,63,52
4500,0.053000,1.759224,0.244053,0.518898,0.493402,471,133,69,0.479960,0.463789,514,70,50


/workspace/stt-split-audio/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/workspace/stt-split-audio/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/workspace/stt-split-audio/venv/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process 

TrainOutput(global_step=5099, training_loss=0.017570176903559522, metrics={'train_runtime': 1142.2369, 'train_samples_per_second': 17.772, 'train_steps_per_second': 4.465, 'total_flos': 6.831284410775477e+18, 'train_loss': 0.017570176903559522, 'epoch': 50.0})

In [21]:
# Save the fine-tuned model
print("training finished")

training finished
